##### Padronizacao

In [ ]:
import pandas as pd

dadosDiadema = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\diadema,-são paulo-air-quality.csv")
dadosItaim = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\itaim-paulista, são paulo-air-quality.csv")
dadosItaquera = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\itaquera,-são paulo-air-quality.csv")
dadosMooca = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\mooca,-são paulo-air-quality.csv")
dadosSantoandre = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\s.andré-capuava,-são paulo-air-quality.csv")
dadosSaoBernardo = pd.read_csv(r"C:\Users\50749314877\Desktop\Git\ProjetoIntegradorSensores\OutrosPoluentes\s.bernardo-centro,-são paulo-air-quality.csv")

In [ ]:
import pandas as pd
import glob

# Dicionário com os dados de latitude e longitude por região
coordenadas = {
    "Sao Bernardo": {"latitude": -23.701934820154197, "longitude": -46.553310586836886, "regiao": "Sao Bernardo"},
    "Santo Andre": {"latitude": -23.658138559401298, "longitude": -46.48984790027418, "regiao": "Santo Andre"},
    "Diadema": {"latitude": -23.685313382167138, "longitude": -46.625954889892164, "regiao": "Diadema"},
    "Itaquera": {"latitude": -23.541638970943424, "longitude": -46.47125736949073, "regiao": "Itaquera"},
    "Itaim Paulista": {"latitude": -23.499440086125986, "longitude": -46.39547580760927, "regiao": "Itaim Paulista"},
    "Mooca": {"latitude": -23.55817427309982, "longitude": -46.60807618968641, "regiao": "Mooca"},
}

# Caminho para todos os arquivos CSV
arquivos_csv = glob.glob("*.csv")  # Ajuste o caminho se necessário

# Processa cada arquivo CSV
for arquivo in arquivos_csv:
    # Lê o arquivo
    df = pd.read_csv(arquivo)
    
    # Extrai a região do nome do arquivo (assumindo que está no nome, ex.: "Santo Andre.csv")
    cidade = arquivo.split(".csv")[0]
    
    # Adiciona latitude, longitude e região
    if cidade in coordenadas:
        df["latitude"] = coordenadas[cidade]["latitude"]
        df["longitude"] = coordenadas[cidade]["longitude"]
        df["regiao"] = coordenadas[cidade]["regiao"]
    else:
        print(f"Região não encontrada no dicionário: {cidade}")
        continue

    # Salva o arquivo atualizado
    df.to_csv(f"{cidade}_atualizado.csv", index=False)

print("Processamento concluído.")

Processamento concluído.


In [ ]:
import pymysql
import csv
from datetime import datetime
import glob

# Conexão com o banco de dados
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='525748',
    database='bd_medidor',
)

coordenadas = {
    "Sao Bernardo": {"latitude": -23.701934820154197, "longitude": -46.553310586836886, "regiao": "Sao Bernardo"},
    "Santo Andre": {"latitude": -23.658138559401298, "longitude": -46.48984790027418, "regiao": "Santo Andre"},
    "Diadema": {"latitude": -23.685313382167138, "longitude": -46.625954889892164, "regiao": "Diadema"},
    "Itaquera": {"latitude": -23.541638970943424, "longitude": -46.47125736949073, "regiao": "Itaquera"},
    "Itaim Paulista": {"latitude": -23.499440086125986, "longitude": -46.39547580760927, "regiao": "Itaim Paulista"},
    "Mooca": {"latitude": -23.55817427309982, "longitude": -46.60807618968641, "regiao": "Mooca"},
}

def convert_to_float(value):
    try:
        return float(value.strip()) if value.strip() else 0.0
    except ValueError:
        return 0.0

def process_csv_file(csv_file_path, connection):
    cidade = csv_file_path.split("/")[-1].split(".csv")[0].replace("-", " ").title()

    if cidade not in coordenadas:
        print(f"Região não encontrada no dicionário: {cidade}")
        return

    latitude = coordenadas[cidade]["latitude"]
    longitude = coordenadas[cidade]["longitude"]
    regiao = coordenadas[cidade]["regiao"]

    with open(csv_file_path, encoding='utf-8') as csv_file:
        csv_data = csv.reader(csv_file)
        next(csv_data)

        cursor = connection.cursor()
        for row in csv_data:
            try:
                data = datetime.strptime(row[0].strip(), '%Y/%m/%d')
                pm25 = convert_to_float(row[1])
                pm10 = convert_to_float(row[2])
                o3 = convert_to_float(row[3])
                no2 = convert_to_float(row[4])
                so2 = convert_to_float(row[5]) if len(row) > 5 else 0.0
                co = convert_to_float(row[6]) if len(row) > 6 else 0.0
                
                if cursor.execute("""
                    INSERT INTO dados_poluentes (data, pm25, pm10, o3, no2, so2, co, latitude, longitude, regiao) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (data, pm25, pm10, o3, no2, so2, co, latitude, longitude, regiao)): 
                    print(f"Inserindo: {data}, {pm25}, {pm10}, {o3}, {no2}, {so2}, {co}, {latitude}, {longitude}, {regiao}")

            except Exception as e:
                print(f"Erro ao processar linha: {row}. Erro: {e}")

        connection.commit()

arquivos_csv = glob.glob("*.csv")
for arquivo in arquivos_csv:
    process_csv_file(arquivo, connection)

connection.close()

#### Dados Sensores

In [ ]:
import pymysql
import csv
from datetime import datetime

# Conexão com o banco de dados
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='senai@134',
    database='bd_medicao'
)

def insert_batch(file_path):
    with open(file_path, encoding='utf-8') as csv_file:
        csv_data = csv.reader(csv_file, delimiter=';')
        next(csv_data)  # Ignorar o cabeçalho

        cursor = connection.cursor()
        batch_size = 1000  # Define o tamanho do lote
        batch = []

        for row in csv_data:
            try:
                # Substituir vírgulas por pontos para valores decimais
                temperatura = float(row[0].replace(',', '.'))
                pressao = float(row[1].replace(',', '.'))
                altitude = float(row[2].replace(',', '.'))

                # Tratar "NULL" e converter para None
                umidade = None if row[3].strip().upper() == "NULL" else int(row[3])
                co2 = None if row[4].strip().upper() == "NULL" else float(row[4].replace(',', '.'))
                tempo_registro = datetime.strptime(row[5], '%d/%m/%Y %H:%M').strftime('%Y-%m-%d %H:%M:%S')
                regiao = row[6]

                # Adicionar os valores ao lote
                batch.append((temperatura, pressao, altitude, umidade, co2, tempo_registro, regiao))

                # Insere o lote quando atinge o tamanho definido
                if len(batch) >= batch_size:
                    cursor.executemany("""
                        INSERT INTO tb_registro (temperatura, pressao, altitude, umidade, co2, tempo_registro, regiao)
                        VALUES (%s, %s, %s, %s, %s, %s, %s)
                    """, batch)
                    connection.commit()
                    batch = []  # Limpa o lote
            except Exception as e:
                print(f"Erro ao processar linha: {row}. Erro: {e}")

        # Insere o restante dos dados
        if batch:
            cursor.executemany("""
                INSERT INTO tb_registro (temperatura, pressao, altitude, umidade, co2, tempo_registro, regiao)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, batch)
            connection.commit()

        print("Importação concluída com sucesso!")

# Caminho para o arquivo CSV
file_path = r''

# Executar a função
insert_batch(file_path)

# Fechar a conexão com o banco de dados
connection.close()